In [25]:
import pandas as pd
import yaml

# Knuth's X algorithm basics

Below are classes that implement elements out of which the sparse matrix used in Knuth's X algorithm will be build. The idea is pretty straightforward - each class is a quite literal impementation of waht his paper describes.

In [26]:
class node:

    def __init__(self):
        self.up = self
        self.down = self
        self.left = self
        self.right = self
        self.COL = None
        self.ROW = None

    def hideInCol(self):
        self.up.down = self.down
        self.down.up = self.up
        self.COL.row_count -= 1
        return self

    def showInCol(self):
        self.up.down = self
        self.down.up = self
        self.COL.row_count += 1
        return self

    def hideInRow(self):
        self.left.right = self.right
        self.right.left = self.left
        return self

    def showInRow(self):
        self.left.right = self
        self.right.left = self
        return self
    
    def deleteUnuseableRow(self):
        cur_cell = self.right 
        while cur_cell != self:
            print(cur_cell.COL.label)
            cur_cell.hideInCol()
            cur_cell = cur_cell.right
        print("done")
        self.hideInCol()
        return

    def hideWholeRow(self, start = None):
        if start == None: # if we just started the loop, don't hide me from my column, but go to my neighbor
            self.right.hideWholeRow(start = self)
            return
        if start != self: # we are not first and not last => hide me and continue
            self.hideInCol()
            self.right.hideWholeRow(start = start)
            return
        return # last possibility start == self => we have looped and can finish

    def showWholeRow(self, start = None):
        if start == None: # if we just started the loop, don't hide me from my column, but go to my neighbor
            self.left.showWholeRow(start = self)
            return
        if start != self: # we are not first and not last => hide me and continue
            self.showInCol()
            self.left.showWholeRow(start = start)
            return
        return # last possibility start == self => we have looped and can finish
    
    def getNeighbor(self, direction, by=0):
        neighbor = {
            'left'  : self.left,
            'right' : self.right,
            'up'    : self.up,
            'down'  : self.down
        }[direction]
        if by > 0:
            return neighbor.getNeighbor(direction, by = by-1)
        return neighbor




In [27]:
class columnHeader(node):

    def __init__(self, label):
        super().__init__()
        self.label = label
        self.row_count = 0
        self.COL = self

    def hideMe(self):
        self.hideInRow()
        cell = self.down
        while cell != self:
            cell.hideWholeRow()
            cell = cell.down
        return self

    def showMe(self):
        self.showInRow()
        cell = self.up
        while cell != self:
            cell.showWholeRow()
            cell = cell.up
        return self

    def appendRow(self, row):
        row.down = self
        row.up = self.up
        self.up.down = row
        self.up = row
        row.COL = self
        self.row_count += 1
        return self

    def addToRoot(self, root):
        root.appendColumn(self)
        return self

In [28]:
class rowHeader(node):
    
    def __init__(self, label):
        super().__init__()
        self.label = label
        self.ROW = self

    def appendColumn(self, col):
        col.right = self
        col.left = self.left
        self.left.right = col
        self.left = col
        col.ROW = self
        return self

    def addToRoot(self, root):
        root.appendRow(self)
        return self


In [29]:
class rootNode(columnHeader, rowHeader):

    def __init__(self):
        node.__init__(self)
        self.row_count = 0
        self.label = None
        self.ROW = self
        self.COL = self

    def addToRoot(self, root):
        pass

# Helper functions

In [30]:
def append_dict(d, e):

    """
    dicrionary.update() returns nothing, but I wanted to chain methods, use it in lambdas etc.
    so this is a wrapper that takes two dicts and retirns the new updated one
    """
    
    r = d.copy()
    r.update(e)
    return r

In [31]:
def check_simple_criterion(sparse_matrix_row_attr, check_attr):

    """Given two sets of attributes (each represented as a dictionary of (dimension name : dimension value) it checks if the attributes match. The idea is:
    - sparse_matrix_row_attr - is the target: the list of attributes to be checked
    - check_attr - is the soruce:  the conditions to check against
    then
    1. if any dim from conditions is missing in target, check cannot be perfomed and defaults to OK
    2. otherwise, for the common dimensions we check:
        a. if all values match - OK
        b. otherwise - FAIL
    """
    
    has_dims = {k for k in sparse_matrix_row_attr}
    needs_dims = {k for k in check_attr}
    overlap_attrs = has_dims & needs_dims
    can_be_checked = len(overlap_attrs) == len(check_attr)

    # not enough attributes present to perform the  check => default to OK
    if not can_be_checked:
        return True

    # check on each dimension OK => all OK
    summary_check_result = { sparse_matrix_row_attr[d] == check_attr[d] for d in overlap_attrs } 

    return summary_check_result == {True} or summary_check_result == {False}



In [32]:
def check_set_of_criteria(sparse_matrix_row_attr, check_set):

    """
    for a single target (sparse_matrix_row_attr) it applies the check across
    a set of different source conditions (check_set). Result is OK <=> each single check is OK.
    To see what 'source' and what 'target' mean in this context, check the description of check_simple_criterion()
    """

    summary_check_results = {
        check_simple_criterion(sparse_matrix_row_attr, item)
        for item in check_set
    }
    
    return summary_check_results == {True}

In [56]:
def dict_to_tupless(dict):

    """
    dictionaries are sometiems cumbersone in addressing, so I will often transform them into a list of tuples
    """

    return [(k, v) for k, v in dict.items()]

# Let's go

## 1. read the riddle file

First we read in the YML fiel describing the riddle. For notes on how to cosntruct such a file, consult the readme.md in this folder

In [33]:
riddle_file = "input_1_einstein.yml"

In [34]:
with open(riddle_file) as f:
    try:
        riddle_text= yaml.safe_load(f)
    except yaml.YAMLError as exc:
        print(exc)

## 2. reshape data

we then reshape the read in data into something easier to handle downstream. This is neither part of Knuth's algorithm, nor critical to how it was implemented here to solve Zebra Riddles. I jsut made a (poor?) design decision to have very copmpact input fil3s, but they need to be reshaped before use.

### 2.1 reshape "simple conditions"

This part cosntructs a list of "simple conditions".

A simple condition is one that is along the lines of

> Person who `Attrribute A` also is `Attrribute B`

Since each characteristic is alwasy comrpised of dimension (e.g. "house color" or "favourite drink") and the value on that dimension ("green" or "milk"), eventually each entry on the list (each "condition") is effectively strucuted as a dictionary:

```python
[
    ...
    {
        "type" : "_is", # fixed value. redundant here, but kept to be easier to align with following data strucutres
        "params" : [ # here order does not matter
            (
                first_attribute_dimension_name,
                first_attribute_value 
            ),
            (
                second_attribute_dimension_name,
                second_attribute_value 
            )
        ]
    }
    ...
]
```

i.e. "The `Spaniard` (`nationality`) owns the `dog` (`pet`)." will be encoded as 

```python
{
    "type" : "_is", 
    [ 
        ('nationality', 'Spanish'),
        ( 'pert',  'dog')
    ]
}

In [ ]:
equalities_list = [ 
    {
        "type" : "_is",
        "params" : dict_to_tupless(is_condition)
    }
    for is_condition in riddle_text
       if len(is_condition) > 1
]

[{'type': '_is', 'params': [('nation', 'Norway'), ('_#', 1)]},
 {'type': '_is', 'params': [('nation', 'England'), ('house_color', 'red')]},
 {'type': '_is', 'params': [('nation', 'Dutch'), ('drink', 'tea')]},
 {'type': '_is', 'params': [('cig', 'cigars'), ('house_color', 'yellow')]},
 {'type': '_is', 'params': [('nation', 'Germany'), ('cig', 'pipe')]},
 {'type': '_is', 'params': [('_#', 3), ('drink', 'milk')]},
 {'type': '_is', 'params': [('cig', 'no_filter'), ('pet', 'birds')]},
 {'type': '_is', 'params': [('nation', 'Sweden'), ('pet', 'dogs')]},
 {'type': '_is', 'params': [('cig', 'menthol'), ('drink', 'beer')]},
 {'type': '_is', 'params': [('house_color', 'green'), ('drink', 'coffee')]}]

### 2.2 reshape "neighbor" conditions

this is the part where we untangle and reshape all condition in the form


> Person who `Attrribute A` lives in a house next to person who `Attrribute B`

The encoding is quite the same as above, with the difference that the `"type" : "_is"` fixed value is replaced by one denoting the neighborly relationship:

* `_left` - in case the person described by firt attribute on the list lives left (house number **less by 1**) of the person described by the second attribute
* `_left` - in case the person described by firt attribute on the list lives right (house number **increased by 1**) of the person described by the second attribute
* `_adj` (adjacent) - in case the two menioned persons live next to each other, but we don't knwo who lives on which side.

Example:

> The green house is immediately to the right of the ivory house.

would be encoded as 

```python
{
    "type" : "_right", 
    ('color', 'green'),
    ('color', 'ivory')
}

```

In [63]:
1 in [1,2,3]

True

In [ ]:
tmp2 = [
    {
        "type" : condition_name,
        "params" : [
            attr
            for item in neighbor_condition[condition_name]
            for attr in dict_to_tupless(item)
        ]
    }
    for neighbor_condition in riddle_text
    for condition_name, condition_params in neighbor_condition.items() \
        if condition_name in ['_left', '_right', '_adj']
]

tmp2

[{'type': '_left',
  'params': [('house_color', 'green'), ('house_color', 'white')]},
 {'type': '_adj', 'params': [('cig', 'light'), ('pet', 'cat')]},
 {'type': '_adj', 'params': [('cig', 'light'), ('drink', 'water')]},
 {'type': '_adj', 'params': [('nation', 'Norway'), ('house_color', 'blue')]},
 {'type': '_adj', 'params': [('house_color', 'yellow'), ('pet', 'horses')]}]

### 2.3 reshape other data

The only other piece of information we can get is the attribute encoded in the question.

the data structure is the same, but we replace `"type" : "_is"` with  `"type" : "_question"` and there is only one attribute called `"condition"`

So the question 

> Now, who drinks water? Who owns the zebra?

woudl be encoded as 

```python
[
    {
        "type" : "_question", 
        "condition" : {"dim" :'drink', "val" : 'water'}
    },
    {
        "type" : "_question", 
        "condition" : {"dim" :'pet', "val" : 'zebra'}
    }
]
```

In [37]:
other_values = [
    {
        "condition" : condition_name,
        "dim": dim_name,
        "val": value
    }
    for other_condition in riddle_text if len(other_condition) == 1
        for condition_name, condition_param_list in other_condition.items() if condition_name == '_question'
    for condtion_param in condition_param_list
    for dim_name, value in condtion_param.items()
]

other_values

[{'condition': '_question', 'dim': 'pet', 'val': 'fish'}]

### 2.4 combine all conditions into one list

In [43]:
# compile a list of all possible attributes

all_attrs = \
    [e["first"] for e in equalities_list] \
    + [e["second"] for e in equalities_list] \
    + [e["first"] for e in neighbors_list] \
    + [e["second"] for e in neighbors_list] \
    + other_values

all_attrs

[{'dim': 'nation', 'val': 'Norway'},
 {'dim': 'nation', 'val': 'England'},
 {'dim': 'nation', 'val': 'Dutch'},
 {'dim': 'cig', 'val': 'cigars'},
 {'dim': 'nation', 'val': 'Germany'},
 {'dim': '_#', 'val': 3},
 {'dim': 'cig', 'val': 'no_filter'},
 {'dim': 'nation', 'val': 'Sweden'},
 {'dim': 'cig', 'val': 'menthol'},
 {'dim': 'house_color', 'val': 'green'},
 {'dim': '_#', 'val': 1},
 {'dim': 'house_color', 'val': 'red'},
 {'dim': 'drink', 'val': 'tea'},
 {'dim': 'house_color', 'val': 'yellow'},
 {'dim': 'cig', 'val': 'pipe'},
 {'dim': 'drink', 'val': 'milk'},
 {'dim': 'pet', 'val': 'birds'},
 {'dim': 'pet', 'val': 'dogs'},
 {'dim': 'drink', 'val': 'beer'},
 {'dim': 'drink', 'val': 'coffee'},
 {'dim': 'house_color', 'val': 'green'},
 {'dim': 'cig', 'val': 'light'},
 {'dim': 'cig', 'val': 'light'},
 {'dim': 'nation', 'val': 'Norway'},
 {'dim': 'house_color', 'val': 'yellow'},
 {'dim': 'house_color', 'val': 'white'},
 {'dim': 'pet', 'val': 'cat'},
 {'dim': 'drink', 'val': 'water'},
 {'dim'

### 2.5 dictionru of dimensions

Once we compiled all known attributes described in the riddle, we cna cosntruct a dictionary of dimentions.

Each entry is the name of the dimension and a set of all it's possible values. An additional dimension named `_#` is added, that containsall possible house positions (1-based indexing)

In [39]:
dicts = {
    i["dim"] : set()
    for i in all_attrs
}

[
    dicts[i["dim"]].add(i["val"])
    for i in all_attrs
]
    
max_items = max(
    [
        len(v)
        for  v in dicts.values()
    ]
)
dicts['_#'] = {
    r+1
    for r in range(max_items)
}

dicts

{'nation': {'Dutch', 'England', 'Germany', 'Norway', 'Sweden'},
 'cig': {'cigars', 'light', 'menthol', 'no_filter', 'pipe'},
 '_#': {1, 2, 3, 4, 5},
 'house_color': {'blue', 'green', 'red', 'white', 'yellow'},
 'drink': {'beer', 'coffee', 'milk', 'tea', 'water'},
 'pet': {'birds', 'cat', 'dogs', 'fish', 'horses'}}

### some more reshaping

this will probalby need to be refacored becasue there's way to much reshaping happening already.

this is jsut a list of all simple constraints, reshapd for easier use later....

In [46]:
simple_guards = [item for item in riddle_text if len(item) !=1] 

simple_guards

[{'nation': 'Norway', '_#': 1},
 {'nation': 'England', 'house_color': 'red'},
 {'nation': 'Dutch', 'drink': 'tea'},
 {'cig': 'cigars', 'house_color': 'yellow'},
 {'nation': 'Germany', 'cig': 'pipe'},
 {'_#': 3, 'drink': 'milk'},
 {'cig': 'no_filter', 'pet': 'birds'},
 {'nation': 'Sweden', 'pet': 'dogs'},
 {'cig': 'menthol', 'drink': 'beer'},
 {'house_color': 'green', 'drink': 'coffee'}]

In [ ]:
x = {"color" : "red", "shape":"square"}
[(k, v) for k,v in x.items()]



[('color', 'red'), ('shape', 'square')]

AttributeError: 'list' object has no attribute 'items'

In [ ]:
class einstein_matrix:

    """
    This is a class to contain and manage all operations related to cosntructing and using the sparse matrix used in Knuth's X algorithm

    Attributes
    ----------
    matrix_root : rootNode
        the root of the matrix, whera ll other nodes are (possibly indirectly) linked
     column_headers_dir : dictionary
        a directory of all column headers created for the matrix, indexed in a way
        that makes lookup easier the lookup happens when rows are linked to columns
        when the matrix is populated
    row_headers_dir : dictionary
        a directory of all row headers created for the matrix.


    """

    def __init__(self):

        self.matrix_root = rootNode()
        self.column_headers_dir = {}
        self.row_headers_dir = []

    def create_row_labels(self, dicts, guards):

        """
        Takes the dictionary of dimentiosn and known simple (not "neighbor") constraints and constructs a list of persons that have allowed attributes along with a text label for the matrix row. The cosntruction is iterative (dimension by dimension, value by value).
        As we iterate, ny combination of attributes that is known to violate the simple conditions will be discarded as soon as it is recognised as invalid.
        For example we migh have already cosntructed the list up to level 2 with entries like
            {'nation' : 'Ukrainian', 'color' : 'green'}
        with further dimensions pending
        If we then go to next dimension for drink an we know that "The Ukrainian drinks tea."
        then from this poin onwards any combinations other than
            {'nation' : 'Ukrainian', 'color' : 'green', 'drink' : 'tea'} 
        Will be discarded (but combinations of other drinks with non-Ukrainian nationality will be kept)
        """
    
        level = 0
        labels = [{}]
        copy_dics = dicts.copy()

        while len(copy_dics) > 0:
            dim = copy_dics.popitem()
            dim_name = dim[0]
            dim_values = dim[1]
            labels = [
                append_dict(label, {dim_name: value})
                for label in labels 
                for value in dim_values
                if check_set_of_criteria(
                    append_dict(label, {dim_name: value}),
                guards)
            ]

        return labels

    def create_rows(self, dicts, guards):

        """
        based on filtered conditions and their row labels - generated by create_row_labels() - this function creates all rowHEader objects, adds them to the matrix's root node and registers each row in the row directory for easier lookup
        """
        
        labels = self.create_row_labels(dicts, guards)

        self.row_headers = [
            {
                "attrs" : l,
                "row_header" :
                    rowHeader(
                        ":".join(
                            [
                                str(i) for i in list(l.values())
                            ]
                        )
                    ).addToRoot(self.matrix_root)
                
            }   for l in labels
        ]

        return self

    def create_columns(self, dicts):

        """
        this creates all the "simple" columns i.e. ones representing the "A person with attribute X (dimenion+value) exists
        """

        self.simple_column_headers = {
            dim :{val : columnHeader(f"{dim}:{val}").addToRoot(self.matrix_root) for val in vals} for dim, vals in dicts.items()
        }

        return self

    def create_neighbor_columns(self, neighbor_params, number_of_houses):

        """
        this function creates columns for the "neighbor" constraints.
        because of how the set cover problems work, it is cosntructed as:
        - in general each neighbor condition is encoded once for each of the "houses". i.e constraint "Z lives next to Y" needs 5 colums if therea re 5 houses.
          - we need one less column for left/right neighbors, because terminal houses are not viable in those cases
          - we need one extra column for "adjacent" neighbor ("proper match flag"), but that's more complex - see the project readme
        - later, when we cosntruct the sparse matrix, we will link correct rows to these columns
          - first neighbor always takes up all columns except for where their second neighbor could be (two in case of "adjacent")
          - second neighbor takes one of the remaining spots. Fot "adjacent" neighbors all can take either spot, but:
            - if it's the proper neightbor listed by the constraint, it also takes the extra "proper match flag" column
            - if it's any other person, it does not take the flag column
        """

        for cond in neighbor_params:
            direction = cond["type"]
            for i in range(number_of_houses):
                house_index = i+1
                label_txt = f"{house_index} : {cond["first"]["dim"]}:{cond["first"]["val"]} {direction} {cond["second"]["dim"]}:{cond["second"]["val"]}"
                label_dict = {
                    "house_index" : house_index,
                    "first_dim" : cond["first"]["dim"],
                    "first_val" : cond["first"]["val"],
                    "second_dim" : cond["second"]["dim"], 
                    "second_val" : cond["second"]["val"]
                }
                header_node = columnHeader(label_txt).addToRoot(self.matrix_root)
                self.neighbor_col_headers.append(
                    {
                        "attrs" : label_dict,
                        "rcol_header" :  header_node
                    }
                )
        return self





SyntaxError: invalid syntax (2317633555.py, line 24)

In [ ]:
em = einstein_matrix()
em.create_columns(dicts) \
   .create_rows(dicts, simple_guards) \
   .create_neighbor_columns(neighbors_list, max_items)
[i for i in em.neighbor_col_headers
if
   True
   # and i["attrs"]["house_index"] == 1
   and i["attrs"]["second_dim"] == "pet"
   and i["attrs"]["second_val"] == "horses"
]


[{'attrs': {'house_index': 1,
   'first_dim': 'house_color',
   'first_val': 'yellow',
   'second_dim': 'pet',
   'second_val': 'horses'},
  'rcol_header': <__main__.columnHeader at 0x7fe7253e5c10>},
 {'attrs': {'house_index': 2,
   'first_dim': 'house_color',
   'first_val': 'yellow',
   'second_dim': 'pet',
   'second_val': 'horses'},
  'rcol_header': <__main__.columnHeader at 0x7fe7253e5cd0>},
 {'attrs': {'house_index': 3,
   'first_dim': 'house_color',
   'first_val': 'yellow',
   'second_dim': 'pet',
   'second_val': 'horses'},
  'rcol_header': <__main__.columnHeader at 0x7fe7253e54f0>},
 {'attrs': {'house_index': 4,
   'first_dim': 'house_color',
   'first_val': 'yellow',
   'second_dim': 'pet',
   'second_val': 'horses'},
  'rcol_header': <__main__.columnHeader at 0x7fe7253e60c0>},
 {'attrs': {'house_index': 5,
   'first_dim': 'house_color',
   'first_val': 'yellow',
   'second_dim': 'pet',
   'second_val': 'horses'},
  'rcol_header': <__main__.columnHeader at 0x7fe7253e5d00>}]

In [ ]:
def search(matrix, solution=[], log=[], k=0):

    # print(f"k : {k}")

    if type(matrix) != rootNode:
        raise TypeError("Must start with matrix root node")

    # solution found if matrix is empty
    if matrix.right == matrix:
        print("ready")
        return prep_solution(solution)

    # select column
    col = matrix.right
    tmp_col = col
    while tmp_col != matrix:
        if tmp_col.row_count < col.row_count:
            col = tmp_col
        tmp_col = tmp_col.right
    print(col.label)
    # hide column 
    col.hideMe()
    # go through its each row
    row = col.down
    print(row.ROW.label)
    while row != col:
        # add row to solution
        solution.append(row)
        log.append(f"select R{row.ROW.label['R']}C{row.ROW.label['C']}={row.ROW.label['#']}")
        # hide row's columns
        cell = row.right
        while cell != row:
            if type(cell) != rowHeader:
                cell.COL.hideMe()
            cell = cell.right
        # go deeper
        result = search(matrix, solution, log, k = k+1)
        if result != None:
            return result
        # uncover columns and remove row from solution
        solution.pop()
        log.append("back")
        cell = row.left
        while cell != row:
            if type(cell) != rowHeader:
                cell.COL.showMe()
            cell = cell.left
        # next row
        row = row.down
    # # uncover column
    col.showMe()
    return None

    